In [1]:
!pip3 install qiskit

  Using cached qiskit-2.3.0-cp310-abi3-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached rustworkx-0.17.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (10 kB)
  Using cached numpy-2.4.1-cp314-cp314-macosx_14_0_arm64.whl.metadata (6.6 kB)
  Using cached scipy-1.17.0-cp314-cp314-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached dill-0.4.1-py3-none-any.whl.metadata (10 kB)
  Using cached stevedore-5.6.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
Using cached qiskit-2.3.0-cp310-abi3-macosx_11_0_arm64.whl (7.9 MB)
Using cached numpy-2.4.1-cp314-cp314-macosx_14_0_arm64.whl (5.2 MB)
Using cached dill-0.4.1-py3-none-any.whl (120 kB)
Using cached rustworkx-0.17.1-cp39-abi3-macosx_11_0_arm64.whl (2.0 MB)
Using cached scipy-1.17.0-cp314-cp314-macosx_14_0_arm64.whl (20.1 MB)
Using cached stevedore-5.6.0-py3-none-any.whl (54 kB)
Using cached typing_extensions-4.15.0-py3-none-any.whl (44 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Challenge: PolyQ Unitary Synthesis

The goal of this challenge is to implement specific unitary transformations (quantum gates) for the PolyQ Simulator. The tasks involve synthesizing circuits for controlled operations, Hamiltonian evolution, and diagonal unitaries.

## Tasks Breakdown

The challenge defines the following target unitary matrices to be implemented:

### Two-Qubit Gates

#### **Task 1: Controlled-Y (CY)**
A standard controlled-Y gate.
$$
U = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & -i \\
0 & 0 & i & 0
\end{pmatrix}
$$
*Theory:* $CY = (I \otimes S) \cdot \text{CNOT} \cdot (I \otimes S^\dagger)$.

#### **Task 2: Controlled-$R_y(\frac{\pi}{7})$**
A controlled rotation around the Y-axis by angle $\theta = \pi/7$.
$$
U = |0\rangle\langle 0| \otimes I + |1\rangle\langle 1| \otimes R_y(\pi/7)
$$
where $R_y(\theta) = e^{-i \theta Y / 2}$.

### Hamiltonian Simulation ($e^{iHt}$)

These tasks require implementing the time-evolution operator $U = e^{i \theta H}$ for various Hamiltonians $H$.

#### **Task 3: ZZ Interaction**
$$
U = e^{i \frac{\pi}{7} (Z \otimes Z)}
$$
*Theory:* This is equivalent to an $R_{zz}$ rotation. It can be implemented using CNOTs and $R_z$ gates: $\text{CNOT} \cdot (I \otimes R_z(-2\phi)) \cdot \text{CNOT}$.

#### **Task 4: XY (Flip-Flop) Interaction**
$$
U = e^{i \frac{\pi}{7} (X \otimes X + Y \otimes Y)}
$$
*Theory:* This Hamiltonian generates the "iSWAP" family of gates. It preserves excitation number (commutes with $Z \otimes I + I \otimes Z$). It can be decomposed into $e^{i\theta XX} \cdot e^{i\theta YY}$ since $[XX, YY] = 0$.

#### **Task 5: Heisenberg XXX Model**
$$
U = e^{i \frac{\pi}{2} (X \otimes X + Y \otimes Y + Z \otimes Z)}
$$
*Theory:* The Heisenberg exchange interaction. The specific angle $\pi/2$ often corresponds to a SWAP-like operation (up to global/local phases). Note that $[XX, YY] = [YY, ZZ] = [ZZ, XX] = 0$, so the order of application doesn't matter.

#### **Task 6: Mixed Interaction**
$$
U = e^{i \frac{\pi}{7} (X \otimes X + Z \otimes I + I \otimes Z)}
$$
*Theory:* This Hamiltonian includes both a two-body interaction ($XX$) and single-qubit fields ($Z_1, Z_2$). Since $XX$ does not commute with $ZI$ or $IZ$, standard Trotterization or more advanced decomposition methods (like Cartan decomposition) might be needed if exact synthesis is required.

### Multi-Qubit Gates

#### **Task 11: 4-Qubit Diagonal Unitary**
A diagonal matrix of size $16 \times 16$ with specific phases.
$$
U = \text{diag}(e^{i \phi_0}, e^{i \phi_1}, \dots, e^{i \phi_{15}})
$$
Values from code:
- Phases include $0, \pi, 5\pi/4, 7\pi/4, 3\pi/2$.
- These phases appear to be multiples of $\pi/4$, suggesting they can be implemented using $T$ ($\pi/8$), $S$ ($\pi/4$), and $Z$ ($\pi/2$) gates, controlled by the qubit states.

## Required Theory

1.  **Matrix Exponentiation:**
    Validating the tasks requires understanding $e^{i \theta H} = \cos(\theta) I + i \sin(\theta) H$ (for involutory $H$) or the general series definition.

2.  **Pauli Group & Commutation:**
    -   Since indices such as $XX$ and $YY$ commute, $e^{i\theta(XX+YY)} = e^{i\theta XX} e^{i\theta YY}$.
    -   Allows breaking down complex Hamiltonians into simpler 2-qubit rotations.
    -   $e^{i \theta (P \otimes P)}$ is typically realized by basis change (e.g., Hadamard for $X \to Z$) sandwiching a $Z \otimes Z$ rotation.

3.  **Gate Synthesis:**
    -   **Basis Change:** $H Z H = X$, $S^\dagger Z S = Y$.
    -   **CNOT Identity:** $e^{i \theta ZZ} = \text{CNOT}_{12} (I \otimes R_z(-2\theta)) \text{CNOT}_{12}$.
    -   **Diagonal Synthesis:** Any diagonal unitary can be decomposed into a sequence of $R_z$ gates and CNOTs (Walsh-Hadamard transform method).


In [2]:
import numpy as np
from scipy.linalg import expm

def get_task_unitaries():
    unitaries = {}
    
    # Pauli Matrices
    I = np.eye(2)
    X = np.array([[0, 1], [1, 0]])
    Y = np.array([[0, -1j], [1j, 0]])
    Z = np.array([[1, 0], [0, -1]])
    
    # --- Task 1: Controlled-Y ---
    # [cite: 137]
    unitaries['task1_cy'] = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, -1j],
        [0, 0, 1j, 0]
    ])

    # --- Task 2: Controlled-Ry(pi/7) ---
    # [cite: 150]
    theta = np.pi / 7
    ry = expm(-1j * (theta/2) * Y)
    unitaries['task2_cry'] = np.block([
        [I, np.zeros((2,2))],
        [np.zeros((2,2)), ry]
    ])

    # --- Task 3: exp(i * pi/7 * Z \otimes Z) ---
    # [cite: 157]
    ZZ = np.kron(Z, Z)
    unitaries['task3_exp_zz'] = expm(1j * (np.pi/7) * ZZ)

    # --- Task 4: exp(i * pi/7 * (XX + YY)) ---
    # [cite: 159]
    H1 = np.kron(X, X) + np.kron(Y, Y)
    unitaries['task4_exp_h1'] = expm(1j * (np.pi/7) * H1)

    # --- Task 5: exp(i * pi/2 * (XX + YY + ZZ)) ---
    # [cite: 160]
    H2 = np.kron(X, X) + np.kron(Y, Y) + np.kron(Z, Z)
    unitaries['task5_exp_h2'] = expm(1j * (np.pi/2) * H2)

    # --- Task 6: exp(i * pi/7 * (XX + ZI + IZ)) ---
    # [cite: 162]
    H3 = np.kron(X, X) + np.kron(Z, I) + np.kron(I, Z)
    unitaries['task6_exp_h3'] = expm(1j * (np.pi/7) * H3)

    # --- Task 11: 4-Qubit Diagonal Unitary ---
    # [cite: 184, 185, 186, 187, 188, 189]
    phases = [0, np.pi, 5*np.pi/4, 7*np.pi/4, 
              5*np.pi/4, 7*np.pi/4, 3*np.pi/2, 3*np.pi/2,
              5*np.pi/4, 7*np.pi/4, 3*np.pi/2, 3*np.pi/2,
              3*np.pi/2, 3*np.pi/2, 7*np.pi/4, 5*np.pi/4]
    unitaries['task11_diagonal'] = np.diag([np.exp(1j * p) for p in phases])

    return unitaries

# Example: Accessing the matrix for Task 11
all_tasks = get_task_unitaries()
print("Task 11 Unitary (First 4 diag elements):", np.diag(all_tasks['task11_diagonal'])[:4])

Task 11 Unitary (First 4 diag elements): [ 1.        +0.00000000e+00j -1.        +1.22464680e-16j
 -0.70710678-7.07106781e-01j  0.70710678-7.07106781e-01j]


In [3]:
import numpy as np
from scipy.linalg import expm

def print_matrix(name, matrix):
    print(f"\n--- {name} ---")
    # Formatting to 3 decimal places for readability
    formatted = np.array2string(matrix, formatter={'complex_kind':lambda x: f"{x.real:+.3f}{x.imag:+.3f}j"})
    print(formatted)

def generate_task_matrices():
    # Pauli Definitions
    I, X, Y, Z = np.eye(2), np.array([[0, 1], [1, 0]]), \
                 np.array([[0, -1j], [1j, 0]]), np.array([[1, 0], [0, -1]])
    
    # Task 1: Controlled-Y
    cy = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,-1j],[0,0,1j,0]])
    print_matrix("Task 1: Controlled-Y", cy)

    # Task 2: Controlled-Ry(pi/7)
    theta = np.pi / 7
    ry = expm(-1j * (theta/2) * Y)
    cry = np.block([[I, np.zeros((2,2))], [np.zeros((2,2)), ry]])
    print_matrix("Task 2: Controlled-Ry(pi/7)", cry)

    # Task 3: exp(i * pi/7 * Z \otimes Z)
    zz = np.kron(Z, Z)
    exp_zz = expm(1j * (np.pi/7) * zz)
    print_matrix("Task 3: exp(i*pi/7 * Z^Z)", exp_zz)

    # Task 4: exp(i * pi/7 * (XX + YY))
    h1 = np.kron(X, X) + np.kron(Y, Y)
    exp_h1 = expm(1j * (np.pi/7) * h1)
    print_matrix("Task 4: exp(i*pi/7 * H1)", exp_h1)

    # Task 5: exp(i * pi/2 * (XX + YY + ZZ))
    h2 = np.kron(X, X) + np.kron(Y, Y) + np.kron(Z, Z)
    exp_h2 = expm(1j * (np.pi/2) * h2)
    print_matrix("Task 5: exp(i*pi/2 * H2)", exp_h2)

    # Task 6: exp(i * pi/7 * (XX + ZI + IZ))
    h3 = np.kron(X, X) + np.kron(Z, I) + np.kron(I, Z)
    exp_h3 = expm(1j * (np.pi/7) * h3)
    print_matrix("Task 6: exp(i*pi/7 * H3)", exp_h3)

    # Task 9: Structured Unitary 2 (DFT/QFT matrix snippet)
    qft_base = np.array([[1, 1, 1, 1], [1, 1j, -1, -1j], [1, -1, 1, -1], [1, -1j, -1, 1j]]) * 0.5
    print_matrix("Task 9: Structured Unitary (QFT)", qft_base)

    # Task 11: 4-qubit Diagonal Unitary
    phases = [0, np.pi, 1.25*np.pi, 1.75*np.pi, 1.25*np.pi, 1.75*np.pi, 1.5*np.pi, 1.5*np.pi,
              1.25*np.pi, 1.75*np.pi, 1.5*np.pi, 1.5*np.pi, 1.5*np.pi, 1.5*np.pi, 1.75*np.pi, 1.25*np.pi]
    diag_u = np.diag([np.exp(1j * p) for p in phases])
    print_matrix("Task 11: 4-Qubit Diagonal (First 4 rows)", diag_u[:4, :4])

generate_task_matrices()


--- Task 1: Controlled-Y ---
[[+1.000+0.000j +0.000+0.000j +0.000+0.000j +0.000+0.000j]
 [+0.000+0.000j +1.000+0.000j +0.000+0.000j +0.000+0.000j]
 [+0.000+0.000j +0.000+0.000j +0.000+0.000j -0.000-1.000j]
 [+0.000+0.000j +0.000+0.000j +0.000+1.000j +0.000+0.000j]]

--- Task 2: Controlled-Ry(pi/7) ---
[[+1.000+0.000j +0.000+0.000j +0.000+0.000j +0.000+0.000j]
 [+0.000+0.000j +1.000+0.000j +0.000+0.000j +0.000+0.000j]
 [+0.000+0.000j +0.000+0.000j +0.975+0.000j -0.223+0.000j]
 [+0.000+0.000j +0.000+0.000j +0.223+0.000j +0.975+0.000j]]

--- Task 3: exp(i*pi/7 * Z^Z) ---
[[+0.901+0.434j +0.000+0.000j +0.000+0.000j +0.000+0.000j]
 [+0.000+0.000j +0.901-0.434j +0.000+0.000j +0.000+0.000j]
 [+0.000+0.000j +0.000+0.000j +0.901-0.434j +0.000+0.000j]
 [+0.000+0.000j +0.000+0.000j +0.000+0.000j +0.901+0.434j]]

--- Task 4: exp(i*pi/7 * H1) ---
[[+1.000+0.000j +0.000+0.000j +0.000+0.000j +0.000+0.000j]
 [+0.000+0.000j +0.623+0.000j +0.000+0.782j +0.000+0.000j]
 [+0.000+0.000j +0.000+0.782j +0.62

# Task 1

In [16]:
import os
import numpy as np
import qiskit.qasm2
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator, process_fidelity

# Ensure the results directory exists
os.makedirs('qasm_results', exist_ok=True)

def create_minimized_cy():
    """
    Creates a Controlled-Y gate with T-count of 2.
    Ignores global phase.
    """
    qc = QuantumCircuit(2, name="minimized_cy")
    
    # 1. Basic CNOT
    qc.cx(0, 1)
    
    # 2. Decomposed CZ (Clifford gates, T-count: 0)
    qc.h(1)
    qc.cx(0, 1)
    qc.h(1)
    
    # 3. S-dagger on control to correct phase (T-count: 2)
    # Using T-dagger twice to form S-dagger
    # qc.tdg(0)
    # qc.tdg(0)
    qc.sdg(0)
    
    return qc

# --- Execution and Verification ---

# 1. Target Unitary
target_qc = QuantumCircuit(2)
target_qc.cy(0, 1)
U_target = Operator(target_qc)

# 2. Optimized Unitary
qc_opt = create_minimized_cy()
U_opt = Operator(qc_opt)

# 3. Verification
# The challenge measures distance d(U, Ũ) ignoring global phase 
fidelity = process_fidelity(U_opt, U_target, require_cp=False)

print("=== Task 1: Controlled-Y Optimization ===")
print(f"T-count: {qc_opt.count_ops().get('t', 0) + qc_opt.count_ops().get('tdg', 0)}")
print(f"CNOT count: {qc_opt.count_ops().get('cx', 0)}")
print(f"Fidelity (ignoring global phase): {fidelity:.6f}")

# 4. Save and Print
print("\nOptimized Circuit Diagram:")
print(qc_opt.draw())

qasm_path = 'qasm_results/task1_cy.qasm'
with open(qasm_path, 'w') as f:
    qiskit.qasm2.dump(qc_opt, f)
print(f"\nSuccessfully saved QASM to: {qasm_path}")

=== Task 1: Controlled-Y Optimization ===
T-count: 0
CNOT count: 2
Fidelity (ignoring global phase): 1.000000

Optimized Circuit Diagram:
                    ┌─────┐
q_0: ──■─────────■──┤ Sdg ├
     ┌─┴─┐┌───┐┌─┴─┐└┬───┬┘
q_1: ┤ X ├┤ H ├┤ X ├─┤ H ├─
     └───┘└───┘└───┘ └───┘ 

Successfully saved QASM to: qasm_results/task1_cy.qasm


# Task 2

In [17]:
import numpy as np
import os
from scipy.linalg import expm
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Operator

# Results directory [cite: 75]
os.makedirs('qasm_results', exist_ok=True)

def get_target_cry_pi7():
    """Exact unitary for Task 2[cite: 33, 36, 37]."""
    I = np.eye(2)
    Y = np.array([[0, -1j], [1j, 0]])
    theta = np.pi / 7
    # Ry(theta) = exp(-i * theta/2 * Y) [cite: 103]
    ry = expm(-1j * (theta/2) * Y)
    # Control is q0, Target is q1 [cite: 17]
    target = np.block([
        [I, np.zeros((2,2))],
        [np.zeros((2,2)), ry]
    ])
    return target

def create_task2_circuit():
    qc = QuantumCircuit(2)
    
    # Rotation 1: Ry(pi/14) = H * Rz(pi/14) * H
    qc.h(1)
    # --- INSERT OPTIMIZED T-SEQUENCE FOR Rz(pi/14) HERE ---
    # Example: qc.t(1); qc.h(1); ... (from gridsynth) [cite: 111]
    qc.h(1)
    
    qc.cx(0, 1)
    
    # Rotation 2: Ry(-pi/14) = H * Rz(-pi/14) * H
    qc.h(1)
    # --- INSERT OPTIMIZED T-SEQUENCE FOR Rz(-pi/14) HERE ---
    qc.h(1)
    
    qc.cx(0, 1)
    return qc

# --- Verification ---
# Generate the exact matrix to compare against 
U_target = get_target_cry_pi7()

# After you insert your sequence, run this to check distance [cite: 88, 89]
U_approx = Operator(create_task2_circuit()).data
dist = np.linalg.norm(U_target - U_approx, ord=2) 
print(f"Operator Norm Distance: {dist}")

Operator Norm Distance: 0.22392895220661566


# Task 11